### Spark

Spark is a platform for cluster computing. Spark lets you spread data and computations over clusters with multiple nodes
(think of each node as a separate computer). Splitting up your data makes it easier to work with very large datasets
because each node only works with a small amount of data.

As each node works on its own subset of the total data, it also carries out a part of the total calculations required,
so that both data processing and computation are performed in parallel over the nodes in the cluster. It is a fact
that parallel computation can make certain types of programming tasks much faster.

When to use Spark? Answering questions like the following is helpful:

- Is my data too big to work with on a single machine?
- Can my calculations be easily parallelized?

Spark's core data structure is the Resilient Distributed Dataset (RDD). RDDs are hard to work with and one usually uses
Spark DataFrame abstraction built on top of RDDs. DataFrames are also more optimized for complicated operations than
RDDs.


#### Basics

In [9]:
from pyspark.sql import SparkSession

# create new spark session if necessary
my_spark = SparkSession.builder.getOrCreate()
print(my_spark)
print(4+9)





13
